In [ ]:
#ResNet
import tensorflow as tf
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up the data directory path containing images and labels
data_dir = "D:\\01.Paul\\AIVI006_IK15HGAE_data_ALL"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

img_size = 224
batch_size = 8

# Define data augmentation techniques
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2,
    validation_split=0.2,
)

# Setup train, validation and test generators
train_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Load the ResNet50 model, pre-trained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=tf.keras.Input(shape=(img_size, img_size, 3)))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Create the top layers to be trained with our data
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_set.class_indices), activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_set,
    epochs=10,
    validation_data=val_set
)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_set)
print("Test accuracy:", test_acc)